<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive**

In [2]:
import os

# Csatoljuk a Google Drive-ot
from google.colab import drive
drive.mount('/content/drive')
!pip install pypng
!pip install Pillow

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.7 MB/s eta 0:00:00


# ***Action***

```
'/content/drive/MyDrive/Images'
```



In [ ]:
from PIL import PngImagePlugin, Image
import os

class PngValidator:
    """PNG fájlok érvényességének ellenőrzésére szolgáló osztály"""

    @staticmethod
    def is_valid_png(file_path):
        """
        Ellenőrzi, hogy a fájl valóban PNG formátumú-e

        Args:
            file_path (str): A vizsgálandó fájl útvonala

        Returns:
            bool: True ha a fájl valóban PNG, False ha nem
        """
        try:
            with Image.open(file_path) as img:
                return isinstance(img, PngImagePlugin.PngImageFile) and img.format == 'PNG'
        except Exception:
            return False

class PngFileHandler:
    """PNG fájlok és mappák kezelésére szolgáló osztály"""

    def __init__(self, directory_path=None):
        """
        Inicializálja az osztályt

        Args:
            directory_path (str, optional): A mappa elérési útja
        """
        self.directory_path = directory_path

    def set_directory_path(self, directory_path):
        """
        Beállítja a feldolgozandó mappa útvonalát

        Args:
            directory_path (str): A mappa elérési útja
        """
        self.directory_path = directory_path

    def get_png_files(self):
        """
        Összegyűjti az érvényes PNG fájlokat a megadott mappából

        Returns:
            list: Érvényes PNG fájlok útvonalainak listája
        """
        if not self.directory_path:
            print("Nincs megadva mappa útvonal!")
            return []

        if not os.path.exists(self.directory_path):
            print(f"A megadott mappa nem létezik: {self.directory_path}")
            return []

        png_files = []
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if PngValidator.is_valid_png(file_path):
                png_files.append(file_path)
            elif filename.lower().endswith('.png'):
                print(f"\nFigyelmeztetés: A fájl '.png' kiterjesztésű, de nem valódi PNG: {filename}")

        return png_files

class MetadataReader:
    """PNG fájlok metaadatainak olvasására szolgáló osztály"""

    @staticmethod
    def read_metadata(file_path):
        """
        Beolvassa egy PNG fájl metaadatait

        Args:
            file_path (str): A PNG fájl útvonala

        Returns:
            dict: A metaadatok szótár formátumban, vagy None hiba esetén
        """
        try:
            with Image.open(file_path) as img:
                metadata = img.text
                if metadata:
                    print(f"\nMetaadatok a következő fájlhoz: {os.path.basename(file_path)}")
                    print("-" * 50)
                    for k, v in metadata.items():
                        print(f"**{k}**:\n{v}\n")
                    return metadata
                else:
                    print(f"\nNem találtunk metaadatokat a következő fájlban: {os.path.basename(file_path)}")
                    return {}

        except FileNotFoundError:
            print(f"A fájl nem található: {file_path}")
            return None
        except Exception as e:
            print(f"Hiba történt a következő fájlnál {os.path.basename(file_path)}: {str(e)}")
            return None

class PngMetadataProcessor:
    """PNG fájlok metaadatainak feldolgozására szolgáló fő osztály"""

    def __init__(self, directory_path=None):
        """
        Inicializálja az osztályt

        Args:
            directory_path (str, optional): A feldolgozandó mappa útvonala
        """
        self.file_handler = PngFileHandler(directory_path)

    def process_directory(self):
        """
        Feldolgozza a megadott mappában található összes PNG fájlt

        Returns:
            dict: A fájlnevek és a hozzájuk tartozó metaadatok szótára
        """
        print(f"\nPNG fájlok keresése a következő mappában: {self.file_handler.directory_path}")
        print("=" * 70)

        png_files = self.file_handler.get_png_files()
        results = {}

        for file_path in png_files:
            metadata = MetadataReader.read_metadata(file_path)
            if metadata:
                results[os.path.basename(file_path)] = metadata

        if not results:
            print("\nNem találtunk érvényes PNG fájlokat a megadott mappában.")

        return results

# Példa használat
if __name__ == "__main__":
    processor = PngMetadataProcessor("/content/drive/MyDrive/Image/")
    all_metadata = processor.process_directory()

    # Összesítés
    if all_metadata:
        print("\nÖsszesítés:")
        print("=" * 70)
        print(f"Összesen {len(all_metadata)} érvényes PNG fájl metaadatait olvastuk be.")